In [1]:
#========================================================================
# MLM(Masked Language Model) 과 NSP(Next Sentence Predict) 로 Further Pre-Train 시키는 예시
# 참고예제 : https://towardsdatascience.com/how-to-train-bert-aaad00533168
#========================================================================

from transformers import BertTokenizer, BertForPreTraining, AdamW, get_linear_schedule_with_warmup
import torch
from myutils import GPU_info, seed_everything, mlogging

device = GPU_info()
print(device)

#seed 설정
seed_everything(111)

#logging 설정
logger =  mlogging(loggername="bertfptnspmlm", logfilname="bertfptnspmlm")

# tokenizer와 모델 로딩
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForPreTraining.from_pretrained('bert-base-multilingual-cased')
model.to(device)

# test Data 불러옴.
# test data는 .으로 구분된 한줄 문자이 아니라. 한줄에 .로구분된 여러문장이 이어진 문장이어야 함
# 예시:'제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
with open('my_data/data.txt', 'r') as fp:
    text = fp.read().split('\n')

logfilepath:bwdataset_2022-03-10.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:bertfpt2_2022-03-10.log


Some weights of BertForPreTraining were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
text[:3]

['제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다.지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.조지아 공과대학교를 졸업하였다.그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.',
 '1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.그의 별명이 "땅콩 농부" 로 알려졌다.1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다.',
 '대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다.조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.']

In [3]:
# NSP를 만들기 위해, .을 기준으로 문장들을 나눈 후 길이를 얻어 둔다.
bag = [item for sentence in text for item in sentence.split('.') if item != '']
bag_size = len(bag)
print(bag_size)

158


In [4]:
bag[14]

'또한 소련과 제2차 전략 무기 제한 협상에 조인했다'

In [5]:
#NSP는 50:50으로 랜덤한 값, 랜덤하지 않은 문장으로 만든다.
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    # 하나의 문장을 읽어와서 .기준으로 나눈다.
    sentences = [sentence for sentence in paragraph.split('.') if sentence != '']
    num_sentences = len(sentences)
    
     # . 기준으로 나눈 문장이 1이상이면..
    if num_sentences > 1:
        # 문장 a 시작번지는 랜덤하게, 해당 문장 이후로 지정
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        # 0.5 이상 랜덤값이면, 연속적인 문장으로 만듬
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)  #label=0이면 연속적
        # 0.5 이하 랜덤값이면  연속적이 아닌 문장으로 만듬
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)  #label=1이면 비연속적

In [6]:
for i in range(3):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')

0
제임스 얼 "지미" 카터 주니어는 민주당 출신 미국 39번째 대통령 이다
---
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다

1
그의 별명이 "땅콩 농부" 로 알려졌다
---
수학의 기초에 대한 위기는 그 당시 수많은 논쟁에 의해 촉발되었으며, 그 논쟁에는 칸토어의 집합론과 브라우어-힐베르트 논쟁이 포함되었다

0
조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다
---
1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다



In [7]:
# 위 NSP 리스트 들을 tokenizer 함

# max_length = 256으로 함, 512 하면 GPU Memory 오류 발생함
max_length = 256
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt',
                   max_length=max_length, truncation=True, padding='max_length')

print(inputs.keys())

print(inputs)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
{'input_ids': tensor([[  101,  9672, 36240,  ...,     0,     0,     0],
        [  101, 21555,  9353,  ...,     0,     0,     0],
        [  101,  9678, 12508,  ...,     0,     0,     0],
        ...,
        [  101,  9638,  9284,  ...,     0,     0,     0],
        [  101, 39671,   117,  ...,     0,     0,     0],
        [  101, 19789,   117,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [8]:
# tokenizer 한 NSP 에 'next_sentence_label' 값(0,1) 추가함
inputs['next_sentence_label'] = torch.LongTensor([label]).T

print(inputs.next_sentence_label[:10])

tensor([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0]])


In [9]:
# MLM 만들기

# labels에는 inputs_id를 복사해서 추가
inputs['labels'] = inputs.input_ids.detach().clone()

print(inputs.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])


In [10]:
# 각 스페셜 tokenid를 구함
CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
UNKtokenid = tokenizer.convert_tokens_to_ids('[UNK]')
PADtokenid = tokenizer.convert_tokens_to_ids('[PAD]')
MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')
print('CLSid:{}, SEPid:{}, UNKid:{}, PADid:{}, MASKid:{}'.format(CLStokenid, SEPtokenid, UNKtokenid, PADtokenid, MASKtokenid))

# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * \
           (inputs.input_ids != CLStokenid) * (inputs.input_ids != SEPtokenid) * \
           (inputs.input_ids != UNKtokenid) * (inputs.input_ids != PADtokenid) * \
           (inputs.input_ids != MASKtokenid)

selection = []
for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )
    
print(selection[:2])

# inputs_ids 에 [MASK] 추가시킴
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = MASKtokenid
    

print(inputs.keys())
print(inputs.input_ids)


CLSid:101, SEPid:102, UNKid:100, PADid:0, MASKid:103
[[5, 11, 22, 27, 39, 41], [3, 28, 29, 39, 40, 43, 57, 59]]
dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])
tensor([[  101,  9672, 36240,  ...,     0,     0,     0],
        [  101, 21555,  9353,  ...,     0,     0,     0],
        [  101,  9678, 12508,  ...,     0,     0,     0],
        ...,
        [  101,  9638,  9284,  ...,     0,     0,     0],
        [  101, 39671,   117,  ...,     0,     0,     0],
        [  101,   103,   117,  ...,     0,     0,     0]])


In [11]:
# dataloader 만듬 
batch_size = 16

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)
    
train_dataset = OurDataset(inputs)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [12]:
# 훈련 시작

from tqdm import tqdm  # for our progress bar

epochs = 2
learning_rate = 2e-5  # 학습률
# optimizer 적용
optimizer = AdamW(model.parameters(), 
                 lr=learning_rate, 
                 eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

# 총 훈련과정에서 반복할 스탭
total_steps = len(train_loader)*epochs

# 스캐줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0, 
                                            num_training_steps=total_steps)
model.zero_grad()# 그래디언트 초기화
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(train_loader, leave=True)
    
    for batch in loop:
        # initialize calculated gradients (from prev step)
        model.zero_grad()# 그래디언트 초기화
    
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        labels = batch['labels'].to(device)
        
        # process
        outputs = model(input_ids = input_ids, 
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        
        # calculate loss for every parameter that needs grad update
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)

        # update parameters
        optimizer.step()
        scheduler.step()  # 학습률 감소
        
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|                                                     | 0/4 [00:00<?, ?it/s]/tmp/ipykernel_98499/3355184558.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 1: 100%|█████████████████████████| 4/4 [00:00<00:00,  4.53it/s, loss=8.12]
